**To use the examples in this chapter, first run the code below to import the right libraries.**

In [ ]:
# =================================
# Imports
# =================================
from PyCh import *
from numpy import random
from dataclasses import dataclass
import math

# 9 Buffers
In the previous chapter we introduced channels. Using channels, entities can be transfered from one process to the next, in a synchronous manner (Sender and receiver perform the communication at exactly the same moment in time, and the communication is instantaneous). In many systems however, processes do not use synchronous communication, they use asynchronous communication instead. Entities (products, packets, messages, simple tokens, and so on) are sent, temporarily stored in a *buffer*, and then received.

In fact, the decoupling of sending and receiving is very important, it allows compensating temporarily differences between the number of items that are sent and received (Under the assumption that the receiver is fast enough to keep up with the sender in general, otherwise the buffer will grow forever or overflow).

For example, consider the exchange of items from a producer process `P` to a consumer process `C` as shown in the figure below. In the unbuffered situation, both processes communicate at the same time.
This means that when one process is (temporarily) faster than the other, it has to wait for the other process before communication can take place.

| Figure 9.1: A producer and a consumer |
-
<img src="figures/8-1.png" width=75%>
<a id='fig:9-1'></a>

As shown in the example below, when the consumer needs more time to process entities than the producer, then the producer will have to wait for the consumer.

In [ ]:
@process
def producer(env, c_out):
    total_t_waiting = 0
    for i in range(5):
        t_ready = env.now
        yield env.execute(c_out.send(i))
        t_waiting = env.now - t_ready
        total_t_waiting = total_t_waiting + t_waiting
        print(f"The producer has sent entity {i} at t = {env.now:.2f}, "
              f"and had to wait {t_waiting} seconds. Total waiting time so far is {total_t_waiting} seconds.")
        yield env.timeout(1)

@process
def consumer(env, c_in):
    while True:
        i = yield env.execute(c_in.receive())
        yield env.timeout(3)

def model():
    env = Environment()
    a = Channel(env)
    P = producer(env, a)
    C = consumer(env, a)
    env.run()

model()

With a buffer in-between, the producer can give its item to the buffer, and continue with its work. Likewise, the consumer can pick up a new item from the buffer at any later time (if the buffer has items). In this simulation tool, buffers are not modeled as channels, they are modeled as additional processes instead. The result is shown in Figure 9.2 below.

| Figure 9.2: A producer and a consumer, with an additional buffer process. |
-
<img src="figures/9-2.png" width=75%>
<a id='fig:9-2'></a>

The producer sends its items synchronously (using channel `a`) to the buffer process. The buffer process keeps the item until it is needed. The consumer gets an item synchronously (using channel `b`) from the buffer when it needs a new item (and one is available).

In manufacturing networks, buffers, in combination with servers, play a prominent role, for buffering items in the network. Various buffer types exist in these networks: buffers can have a finite or infinite capacity, they have an input/output discipline, for example a first-out queuing discipline or a priority-based discipline. Buffers can store different kinds of items, for example, product-items, information-items, or a combination of both. Buffers may also have sorting facilities, etc.

In this chapter some buffer types are described, and with the presented concepts numerous types of buffer can be designed by the engineer. First a simple buffer process with one buffer position is presented, followed by more advanced buffer models.
The producer and consumer processes are not discussed in this chapter.

## 9.1 A one-place buffer
A buffer usually has a receiving channel and a sending channel, for receiving and sending items. A buffer, buffer `B1`, is presented in Figure 9.3.

| Figure 9.3: A 1-place buffer. |
-
<img src="figures/9-3.png" width=75%>
<a id='fig:9-3'></a>

The simplest buffer is a one-place buffer, for buffering precisely one item. A one-place buffer is shown below.  `c_in` and `c_out` are the receiving and sending channels. Entity `i` is buffered in the process. A buffer receives an item, stores the item, and sends the item to the next process, if the next process is willing to receive the item. The buffer is not willing to receive a second item, as long as the first item is still in the buffer. 

In [ ]:
@process
def buffer(env, c_in, c_out):
    while True:
        i = yield env.execute(c_in.receive())
        yield env.execute(c_out.send(i))

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B1 = buffer(env, a, b)
    C = consumer(env, b)
    env.run()

model()

---
A two-place buffer can be created, by using the one-place buffer process twice. A two-place buffer is depicted below.

| Figure 9.4: A 2-place buffer. |
-
<img src="figures/9-4.png" width=75%>
<a id='fig:9-4'></a>

A two-place buffer is shown below. In the two-place buffer, processes `B1` and `B2` buffer maximal two items.
If each buffer process contains an item, a third item has to wait in front of process `B1`. 

Note that `buffer_2_place` is not a process but a submodel, because it does not yield any events by itself

In [ ]:
def buffer_2_place(env, c_in, c_out):
    c_B1B2 = Channel(env)
    B1 = buffer(env, c_in, c_B1B2)
    B2 = buffer(env, c_B1B2, c_out)
    
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B = buffer_2_place(env, a, b)
    C = consumer(env, b)
    env.run()

model()

---
This procedure can be extended to create even larger buffers. Another, more preferable manner however, is to describe a buffer in a single process by using a `select` statement and a list for storage of the items. Such a buffer is discussed in the next section.

## 9.2 A single process buffer
An informal description of the process of a buffer, with an arbitrary number of stored items, is the following:
- If the buffer has space for an item, *and* can receive an item from another process via channel `a`,
    the buffer process receives that item, and stores the item in the buffer.
- If the buffer contains at least one item, *and* the buffer can send that item to another process via
    channel `b`, the buffer process sends that item, and removes that item from the buffer.
- If the buffer can both send and receive a value, the buffer process selects one of the two possibilities (in a non-deterministic manner).
- If the buffer can not receive an item, and can not send an item, the buffer process waits.


Using the select function we learned in the previous chapter, we can create a buffer process, which can simultaneously send and receive items. An example of a finite buffer with `N` capacity is shown below. In the model of the buffer, list `xs` is used for storing the received items.

This buffer can only send when there are items in the buffers (the guard for sending is `len(xs)>0`), and can only receive when the buffer is not yet full (the guard for receiving is `len(xs)<N`). The buffer then either sends or receives an item:

    x = yield env.select(sending, receiving)


Next to the sending and receiving of items (to and from the buffer process) is the question of how to order the stored items.
A common form is the *first-in first-out* (fifo) queuing discipline. Items that enter the buffer first (first-in) also leave first (first-out), the order of items is preserved by the buffer process. Our buffer is has such a fifo policy.

If an item `x` is receive, it is added at the rear of list:

    if selected(receiving):
            xs = xs + [x]
    
If an item `x` is receive, it is removed from the start of the list:
    
    if selected(sending):
            xs = xs[1:]

The model of the finite buffer is shown below:

In [ ]:
# Finite buffer model
@process
def buffer(env, c_in, c_out, N):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None  # If there is an item in the buffer
        receiving = c_in.receive()  if len(xs)<N else None  # If the buffer is not yet full
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]
        print(f"The buffer contains {len(xs)} item(s)")

In [ ]:
# Rest of model
@process
def producer(env, c_out):
    for x in range(5):
        yield env.execute(c_out.send(x))
        yield env.timeout(1)

@process
def consumer(env, c_in):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(3)

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B = buffer(env, a, b, 3)
    C = consumer(env, b)
    env.run()

model()

---
Similarly, a buffer with an infinite capacity can be written as following:

In [ ]:
# Infinite buffer model (FIFO)
@process
def buffer(env, c_in, c_out):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]
        print(f"The buffer contains {len(xs)} item(s)")

In [ ]:
# Rest of model
@process
def producer(env, c_out):
    for x in range(5):
        yield env.execute(c_out.send(x))
        yield env.timeout(1)

@process
def consumer(env, c_in):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(3)

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B = buffer(env, a, b)
    C = consumer(env, b)
    env.run()

model()

---
A first-in first-out buffer is also called a *queue*, while a first-in last-out (*lifo*) buffer is called a *stack*.
A lifo buffer puts the last received item at the head of the list, and gets the first item from the list.

    if selected(receiving):
            xs = [x] + xs

A model of a lifo buffer is:

In [ ]:
# Lifo buffer model
@process
def buffer(env, c_in, c_out):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None 
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = [x] + xs   # items are placed on top of the stack
        if selected(sending):
            xs = xs[1:]
        print(f"The buffer contains {len(xs)} item(s)")

In [ ]:
# Rest of model
@process
def producer(env, c_out):
    for x in range(5):
        yield env.execute(c_out.send(x))
        yield env.timeout(1)

@process
def consumer(env, c_in):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(3)

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B = buffer(env, a, b)
    C = consumer(env, b)
    env.run()

model()

## 9.3 A token buffer
In the next example, signals are buffered instead of items.
The buffer receives and sends 'empty' items or *tokens*.
Counter variable `w` denotes the difference of the number of tokens received and the number of tokens sent.
If the buffer receives a token, counter `w` is incremented; if the buffer sends a token, counter `w` is decremented.
If the number of tokens sent is less than the number of tokens received, there are tokens in the buffer, and `w > 0`.
A receiving channel `a` is defined for receiving tokens.
A sending channel `b` is defined for sending tokens.
The buffer becomes:

In [ ]:
# Token buffer model
@process
def buffer(env, c_in, c_out):
    w = 0;
    while True:
        sending = c_out.send() if w>0 else None 
        receiving = c_in.receive()
        yield env.select(sending, receiving)
        if selected(receiving):
            w = w + 1
        if selected(sending):
            w = w - 1
        print(f"The buffer contains {w} token(s)")

In [ ]:
# Rest of model
@process
def producer(env, c_out):
    for i in range(5):
        yield env.execute(c_out.send())
        yield env.timeout(1)

@process
def consumer(env, c_in):
    while True:
        yield env.execute(c_in.receive())
        yield env.timeout(3)

        
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B = buffer(env, a, b)
    C = consumer(env, b)
    env.run()

model()

## 9.4 A priority buffer

A buffer for items with different priority is described in this section. An item has a high priority or a normal priority.
Items with a high priority should leave the buffer first.

In this example, an item is a dataclass with a field `prio`, denoting the priority, `0` for high priority, and `1` for normal priority:

In [ ]:
@dataclass
class Item:
    prio: int = 0

In the model the received items are, on the basis of the value of the `prio`-field in the item,
stored in one of two lists: one list for 'high' items and one list for 'normal' items.
The discipline of the buffer is that items with a high priority leave the buffer first.
For the storage of items, two lists are used: a list for high priority items and a list for normal priority items.
The two lists are described by a list of two lists `xs`:

    xs = [ [], [] ];
    
List `xs[0]` contains the high priority items, `xs[1]` the normal priority items. The behavior of the priority buffer is as following:
- Received items `x` are stored in `xs[x.prio]` by the statement `xs[x.prio] = xs[x.prio] + [x]`.
- If the list of high priority items is not empty (`len(xs[0])>0`), items with high priority are sent. The first element in list `xs[0]` is element `xs[0][0]`.
- If there are no high priority items (`len(xs[0])==0`), and there are normal priority items (`len(xs[1])>0`), the first element of list `xs[1]`, element `xs[1][0]`, is sent.

The model for the priority buffer is shown below:

In [ ]:
# Priority buffer model
@process
def buffer(env, c_in, c_out):
    xs = [ [], [] ];
    while True:
        sending_prio   = c_out.send(xs[0][0]) if len(xs[0])>0 else None
        sending_normal = c_out.send(xs[1][0]) if (len(xs[1])>0 and len(xs[0])==0) else None
        receiving = c_in.receive()
        x = yield env.select(sending_prio, sending_normal, receiving)
        if selected(receiving):
            xs[x.prio] = xs[x.prio] + [x]
        if selected(sending_prio):
            xs[0] = xs[0][1:]
        if selected(sending_normal):
            xs[1] = xs[1][1:]
        print(f"The buffer contains {len(xs[1])} normal item(s) and {len(xs[0])} priority items")

In [ ]:
# Rest of model
@dataclass
class Item:
    prio: int = 0

@process
def producer(env, c_out):
    random_prio = lambda: random.randint(2)  # priority is assigned randomly 
    for i in range(10):
        x = Item(prio = random_prio())
        yield env.execute(c_out.send(x))
        yield env.timeout(1)

@process
def consumer(env, c_in):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(3)

        
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B = buffer(env, a, b)
    C = consumer(env, b)
    env.run()

model()

## 9.5 Exercises

### Exercise 9.5.1 
 To study product flow to and from a factory, a setup as shown in Figure 9.5 is created. `F` is the factory being studied, generator `G` sends products into the factory, and exit process `E` retrieves finished products. The factory is tightly controlled by controller `C` that sends a signal to `G` or `E` before a product may be moved.

| Figure 9.5: A controlled factory. |
-
<img src="figures/9-5.png" width=100%>
<a id='fig:9-5'></a>

 The model of the system (excluding the factory model) is as follows:

In [ ]:
@process
def Generator(env, c_out, c_signal, N):
    for i in range(N):
        yield env.execute(c_signal.receive())
        yield env.execute(c_out.send(i))
        
@process
def Exit(env, c_in, c_signal):
    while True:
        yield env.execute(c_signal.receive())
        x = yield env.execute(c_in.receive())
        print(f"The Exit received {x}")
        
@process
def Controller(env, c_signal_gen, c_signal_exit, low, high):
    count = 0
    while True:
        while count < high:
            yield env.execute(c_signal_gen.send())
            count = count+1
        while count > low:
            yield env.execute(c_signal_exit.send()) 
            count = count-1
    
def model(low, high, N):
    env = Environment()
    sg = Channel(env)
    se = Channel(env)
    gf = Channel(env)
    fe = Channel(env)
    G = Generator(env, gf, sg, N)
    F = Factory(env, gf, fe)
    E = Exit(env, fe, se)
    C = Controller(env, sg, se, low, high)
    env.run()

---
**A.** As a model of the factory, use a FIFO buffer process. Run the simulation, and check whether all products are received by the exit process.

In [ ]:
# 1a: FIFO factory
@process
def Factory(env, c_in, c_out):
    ...

In [ ]:
# 1a: execute the model
model(low=0, high=1, N=10)

---
**B.** Change the control policy to `low = 1` and `high = 4`. Predict the outcome, and verify with simulation.

Predicted outcome: ...

In [ ]:
# 1b: execute the model
model(low=1, high=4, N=10)

---
**C.** The employees of the factory propose to stack the products in the factory to reduce the amount of space needed for buffering. Replace the factory process with a LIFO buffer process, run the experiments again, first with `low = 0` and `high = 1` and then with `low = 1` and `high = 4`.

In [ ]:
# 1a: LIFO factory
@process
def Factory(env, c_in, c_out):
    ...

In [ ]:
# 1c: execute the model
model(low=0, high=1, N=10)

In [ ]:
# 1c: execute the model
model(low=1, high=4, N=10)

---
**D.** You will notice that some products stay in the factory forever. Why does that happen? How should the policy be changed to ensure all products eventually leave the factory?

Answer: ...

In [ ]:
model(....)

## 9.6 Answers to exercises

### Answer to 9.5.1

<details>
    <summary>[Click for the answer to 9.5.1]</summary>

**A.** All products reach the exit process. The factory model is:

```python
@process
def Factory(env, c_in, c_out):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]
```

Execute the model with:

```python
model(low=0, high=1, N=10)
```

---
**B.** Not all products reach the exit process. The reason is that the parameter low is 1. This parameters determines the minimum number of products that should be in the factory.

```python
model(low=1, high=4, N=10)
```

--- 
**C.**  Replace the factory model with the LIFO buffer model below. 

```python
@process
def Factory(env, c_in, c_out):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = [x] + xs 
        if selected(sending):
            xs = xs[1:]
```
- When simulating with `low = 0` and `high = 1`, all products reach the exit, and in the right order.
```python
model(low=0, high=1, N=10)
```

- When simulating with `low = 1` and `high = 4`, not all products reach the exit, and the order of the products is changed.
```python
model(low=1, high=4, N=10)
```
---
**D.** The first condition is that `low=0`, the second condition is that the generator only generates a multiple of `high` in products. If both conditions are satisfied all products will reach the exit.

```python
model(low=0, high=4, N=12)
```
</details>

